In [ ]:
%pip install cpf-generator
%pip install faker

import pandas as pd
import numpy as np
import random
import datetime
from faker import Faker
from cpf_generator import CPF


In [ ]:
import pandas as pd
import numpy as np
import random
import datetime
from faker import Faker
from cpf_generator import CPF


In [ ]:
# Gerador FAKE
Gerador_Faker = Faker( locale='pt_BR' )
Gerador_CPF = CPF.generate()


# Gerar o RG
# Sequencia 9 digitos + Estado de Origem

def Gerador_RG():

  # Cria uma sequencia de 9 digits
  Numero_RG = str( random.randint(100000000, 999999999) )

  # Estados
  Estado_Origem = str( random.randint(0, 9) )

  # Juntar ambos
  Numero = f'{Numero_RG}-{Estado_Origem}'

  return Numero

In [ ]:
#Variaveis de apoio

Contador = 10

id_contador = 0 

# Dados sinteticos
Dados_Loop = []

for x in range(Contador):

 # GErar o CPF para cada interação
 Gerador_CPF = CPF.generate()

 Dados = {
   'ID' : id_contador + 1,
   'Nome' : Gerador_Faker.name(),
   'Data Nascimento' : Gerador_Faker.date_of_birth(minimum_age=18, maximum_age=50),
   'CPF' : Gerador_CPF,
   'RG' : Gerador_RG(),
   'Empresa' : random.choice(['AZURE', 'AWS', 'GCP']),   
   'Data Admissão' : Gerador_Faker.date_of_birth(minimum_age=2, maximum_age=5),   
   'Departamento' : random.choice( ['Operações', 'Adm', 'Tecnologia'] )
 }

 id_contador += 1

 Dados_Loop.append(Dados)


df = spark.createDataFrame(Dados_Loop)

display(df)


CPF,Data Admissão,Data Nascimento,Departamento,Empresa,ID,Nome,RG
00641648456,2018-06-13,2000-02-03,Adm,GCP,1,Ana Sophia das Neves,630872107-7
79317942148,2020-05-29,1988-04-25,Operações,GCP,2,Igor Carvalho,163194938-1
89681176839,2017-07-31,1979-06-01,Adm,AZURE,3,Benjamin Souza,397306056-1
94875296835,2018-11-11,1972-09-28,Tecnologia,GCP,4,Dr. Cauã Cardoso,186415686-7
84082627855,2018-01-24,1989-01-27,Tecnologia,GCP,5,Thomas Fernandes,153735113-5
16712424321,2021-03-21,1983-03-04,Adm,AZURE,6,João Pedro Pires,660070999-1
84827815453,2017-11-15,1988-03-11,Operações,AZURE,7,Camila Costa,653613411-5
43550750102,2020-07-27,1978-11-17,Tecnologia,AZURE,8,Alexia Farias,394241627-8
02834633177,2019-03-20,1996-11-19,Adm,GCP,9,Stella Ferreira,965541093-9
57928754800,2017-12-01,1998-09-13,Tecnologia,AZURE,10,Danilo Novaes,862883541-5


In [ ]:
dbutils.fs.mount(
 source = "wasbs://projetodados@storageprojetoengdados.blob.core.windows.net",
 mount_point = "/mnt/projetodados", 
 extra_configs = {"fs.azure.account.key.storageprojetoengdados.blob.core.windows.net":"L/NL7PxKQ88pOrYwJT1vx+ryH79wo/4TUKN0Ub3yppvzn9DRRH3L4nn7cuPj+fDVeAd4CKrtqX66+AStETjNvg=="}
 )

In [ ]:
df.coalesce(1).write.format("csv")\
         .mode("overwrite")\
         .option("header", "True")\
         .option("inferSchema","True")\
         .save("/mnt/projetodados/raw/faker/")


In [ ]:
 def mover_arquivo(extensao, origem, destino, novo_nome):
     files = dbutils.fs.ls(origem)
 
     for i in range(0, len(files)):
         file = files[i].name
        
         if extensao in file:
            dbutils.fs.cp(files[i].path, destino + novo_nome + extensao)
            print('Origem é: ' + file + ' ' + 'Destino é: ' + destino + novo_nome + extensao)
 

In [ ]:
mover_arquivo('.csv', '/mnt/projetodados/raw/faker/', '/mnt/projetodados/processing/faker_process/', 'faker')

Origem é: part-00000-tid-8472187731717505212-f0b8f5b5-5990-47ec-a455-98a367a468e7-24-1-c000.csv Destino é: /mnt/projetodados/processing/faker_process/faker.csv


In [ ]:
df_new = spark.read.format("csv")\
             .option("header", "True")\
             .option("inferSchema","True")\
             .load("/mnt/projetodados/processing/faker_process/faker.csv")


In [ ]:
display(df_new)

CPF,Data Admissão,Data Nascimento,Departamento,Empresa,ID,Nome,RG
641648456,2018-06-13,2000-02-03,Adm,GCP,1,Ana Sophia das Neves,630872107-7
79317942148,2020-05-29,1988-04-25,Operações,GCP,2,Igor Carvalho,163194938-1
89681176839,2017-07-31,1979-06-01,Adm,AZURE,3,Benjamin Souza,397306056-1
94875296835,2018-11-11,1972-09-28,Tecnologia,GCP,4,Dr. Cauã Cardoso,186415686-7
84082627855,2018-01-24,1989-01-27,Tecnologia,GCP,5,Thomas Fernandes,153735113-5
16712424321,2021-03-21,1983-03-04,Adm,AZURE,6,João Pedro Pires,660070999-1
84827815453,2017-11-15,1988-03-11,Operações,AZURE,7,Camila Costa,653613411-5
43550750102,2020-07-27,1978-11-17,Tecnologia,AZURE,8,Alexia Farias,394241627-8
2834633177,2019-03-20,1996-11-19,Adm,GCP,9,Stella Ferreira,965541093-9
57928754800,2017-12-01,1998-09-13,Tecnologia,AZURE,10,Danilo Novaes,862883541-5


In [ ]:
df_new.coalesce(1)\
      .write\
      .format("parquet")\
      .mode("overwrite")\
      .save("/mnt/projetodados/processing/faker_process_parquet/")

In [ ]:
mover_arquivo('.parquet', '/mnt/projetodados/processing/faker_process_parquet/', '/mnt/projetodados/refined/faker_curated/', 'tb_faker')

Origem é: part-00000-tid-6328520085202957070-2c64ecd8-c44c-4342-b456-ff36f07266eb-28-1-c000.snappy.parquet Destino é: /mnt/projetodados/refined/faker_curated/tb_faker.parquet


In [ ]:
df_parquet = spark.read.format("parquet")\
                       .load("/mnt/projetodados/refined/faker_curated/tb_faker.parquet")

In [ ]:
df_parquet.createOrReplaceTempView("view_df_parquet")

In [ ]:
df_result = spark.sql("select * from view_df_parquet") 

In [ ]:
display(df_result)

CPF,Data Admissão,Data Nascimento,Departamento,Empresa,ID,Nome,RG
641648456,2018-06-13,2000-02-03,Adm,GCP,1,Ana Sophia das Neves,630872107-7
79317942148,2020-05-29,1988-04-25,Operações,GCP,2,Igor Carvalho,163194938-1
89681176839,2017-07-31,1979-06-01,Adm,AZURE,3,Benjamin Souza,397306056-1
94875296835,2018-11-11,1972-09-28,Tecnologia,GCP,4,Dr. Cauã Cardoso,186415686-7
84082627855,2018-01-24,1989-01-27,Tecnologia,GCP,5,Thomas Fernandes,153735113-5
16712424321,2021-03-21,1983-03-04,Adm,AZURE,6,João Pedro Pires,660070999-1
84827815453,2017-11-15,1988-03-11,Operações,AZURE,7,Camila Costa,653613411-5
43550750102,2020-07-27,1978-11-17,Tecnologia,AZURE,8,Alexia Farias,394241627-8
2834633177,2019-03-20,1996-11-19,Adm,GCP,9,Stella Ferreira,965541093-9
57928754800,2017-12-01,1998-09-13,Tecnologia,AZURE,10,Danilo Novaes,862883541-5
